In [2]:
from pandas import DataFrame, concat
from pickle import load
from datetime import datetime 
from numpy import append

In [ ]:
{
    "Levy":1399,
    "Manufacturer":"LEXUS",
    "Model":"RX 450",
    "Prod_year":2010,
    "Category": "jeep", 
    "Leather_interior": "Yes",
    "Fuel_type": "Hybrid", 
    "Engine_volume": 3.5, 
    "Mileage": 186005,
    "Cylinders": 6.0, 
    "Gear_box_type": "Automatic", 
    "Drive_wheels": "4×4",  
    "Wheel": "Left wheel", 
    "Color": "Silver",
    "Airbags": 12
}

In [176]:
car_dict={
    'Levy':1399,
    'Manufacturer':'LEXUS',
    'Model':'RX 450',
    'Prod_year':2010,
    'Category': 'jeep', 
    'Leather_interior': 'Yes',
    'Fuel_type': 'Hybrid', 
    'Engine_volume': 3.5, 
    'Mileage': 186005,
    'Cylinders': 6.0, 
    'Gear_box_type': 'Automatic', 
    'Drive_wheels': '4×4',  
    'Wheel': 'Left wheel', 
    'Color': 'Silver',
    'Airbags': 12
}

# Plan for model input

### Input Handling

- **Levy** : input as a number, if not provided use **0** as default value
- **Engine Volume**  : numerical input
- **Mileage** : numerical value

### Feature Extraction
- Generate a new feature **Age_Of_Car** from Prod_year

### Feature Removal 
- Remove Features : **ID**, **Doors**lor : 16

### Categorical Encoding
- **df_object** : { 'Manufacturer', 'Model',
  'Category', 'Leather interior',
  'Fuel type', 'Gear box type',
  'Drive wheels', 'Wheel', 'Color' }

### Feature Scaling
- **Levy**
- **Engine Volume**
- **Mileage**
- **Age_Of_Car**

-------------------------------

In [ ]:
# Transform dict to DataFrame and add Age_of_car 
df2 = DataFrame([car_dict]) 
df2

,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Wheel,Color,Airbags
0,1399,LEXUS,RX 450,2010,jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4×4,Left wheel,Silver,12


In [ ]:
df2['Age_of_Car'] = datetime.now().year - df2['Prod_year']
# df2 = df2.drop('Prod_year',axis=1)

In [183]:
df2

,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Wheel,Color,Airbags,Age_of_Car
0,1399,LEXUS,RX 450,2010,jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4×4,Left wheel,Silver,12,15


In [ ]:
# label_encoder_columns = ['Manufacturer', 'Model', 'Category', 'Leather interior', 
#                           'Fuel type', 'Gear box type', 'Drive wheels', 'Wheel', 'Color' ]

# with open('label_encoders.pkl', 'rb') as f:
#     label_encoders = pickle.load(f)  
    
# for col in label_encoder_columns:
#     label = label_encoders[col]
#     df2[col] = label.transform(df2[col])

In [ ]:
# Label Encoding

label_encoder_columns = ['Manufacturer', 'Model', 'Category', 
                         'Fuel_type', 'Color','Leather_interior','Wheel']

with open('label_encoders.pkl', 'rb') as f:
    label_encoders = load(f)

for col in label_encoder_columns:
    if col in label_encoders:  
        label = label_encoders[col]
        
        unseen_values = set(df2[col]) - set(label.classes_)
        if unseen_values:
            label.classes_ = append(label.classes_, list(unseen_values))  

        df2[col] = label.transform(df2[col])
    else:
        print(f"Warning: No encoder found for column {col}")

In [186]:
df2

,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,Gear_box_type,Drive_wheels,Wheel,Color,Airbags,Age_of_Car
0,1399,25,953,2010,11,1,2,3.5,186005,6.0,Automatic,4×4,0,12,12,15


In [187]:
mapping = {category : index for index, category in enumerate(label_encoders['Color'].classes_)}
print(mapping)

{'Beige': 0, 'Black': 1, 'Blue': 2, 'Brown': 3, 'Carnelian red': 4, 'Golden': 5, 'Green': 6, 'Grey': 7, 'Orange': 8, 'Pink': 9, 'Purple': 10, 'Red': 11, 'Silver': 12, 'Sky blue': 13, 'White': 14, 'Yellow': 15}


In [ ]:
# Feature Scaling
Scaling_feature = ['Levy', 'Engine_volume','Mileage', 'Age_of_Car']
with open ('scaler.pkl', 'rb') as file : 
    scaler = load(file)
df2[Scaling_feature] = scaler.transform(df2[Scaling_feature])

--------------------------------

In [ ]:
one_hot_columns = ['Gear_box_type','Drive_wheels']

with open('One_Hot_Encoder.pkl', 'rb') as f:
    One_Hot_Encoder = load(f)

In [191]:
ohe_encoded_data = One_Hot_Encoder.transform(df2[one_hot_columns])

In [ ]:
ohe_encoded_data_df = DataFrame(ohe_encoded_data, columns=One_Hot_Encoder.get_feature_names_out(one_hot_columns), index=df2.index)
df3 = concat([df2, ohe_encoded_data_df], axis=1)
df3.drop(columns=one_hot_columns, inplace=True)

---------------------------------

In [194]:
df3.head()

,Levy,Manufacturer,Model,Prod_year,Category,Leather_interior,Fuel_type,Engine_volume,Mileage,Cylinders,...,Color,Airbags,Age_of_Car,Gear_box_type_Automatic,Gear_box_type_Manual,Gear_box_type_Tiptronic,Gear_box_type_Variator,Drive_wheels_4x4,Drive_wheels_Front,Drive_wheels_Rear
0,3.915291,25,953,2010,11,1,2,4.151722,0.781212,6.0,...,12,12,0.1519,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [195]:
df3.columns

Index(['Levy', 'Manufacturer', 'Model', 'Prod_year', 'Category',
       'Leather_interior', 'Fuel_type', 'Engine_volume', 'Mileage',
       'Cylinders', 'Wheel', 'Color', 'Airbags', 'Age_of_Car',
       'Gear_box_type_Automatic', 'Gear_box_type_Manual',
       'Gear_box_type_Tiptronic', 'Gear_box_type_Variator', 'Drive_wheels_4x4',
       'Drive_wheels_Front', 'Drive_wheels_Rear'],
      dtype='object')

In [196]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Levy                     1 non-null      float64
 1   Manufacturer             1 non-null      int32  
 2   Model                    1 non-null      int32  
 3   Prod_year                1 non-null      int64  
 4   Category                 1 non-null      int32  
 5   Leather_interior         1 non-null      int32  
 6   Fuel_type                1 non-null      int32  
 7   Engine_volume            1 non-null      float64
 8   Mileage                  1 non-null      float64
 9   Cylinders                1 non-null      float64
 10  Wheel                    1 non-null      int32  
 11  Color                    1 non-null      int32  
 12  Airbags                  1 non-null      int64  
 13  Age_of_Car               1 non-null      float64
 14  Gear_box_type_Automatic  1 non

--------------------------

In [198]:
order_in_fit = ['Levy', 'Prod_year', 'Engine_volume', 'Mileage', 'Cylinders', 'Airbags',
       'Age_of_Car', 'Manufacturer', 'Model', 'Category', 'Fuel_type', 'Color',
       'Leather_interior', 'Wheel', 'Gear_box_type_Automatic',
       'Gear_box_type_Manual', 'Gear_box_type_Tiptronic',
       'Gear_box_type_Variator', 'Drive_wheels_4x4', 'Drive_wheels_Front',
       'Drive_wheels_Rear']

df3 = df3[order_in_fit]

In [199]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Levy                     1 non-null      float64
 1   Prod_year                1 non-null      int64  
 2   Engine_volume            1 non-null      float64
 3   Mileage                  1 non-null      float64
 4   Cylinders                1 non-null      float64
 5   Airbags                  1 non-null      int64  
 6   Age_of_Car               1 non-null      float64
 7   Manufacturer             1 non-null      int32  
 8   Model                    1 non-null      int32  
 9   Category                 1 non-null      int32  
 10  Fuel_type                1 non-null      int32  
 11  Color                    1 non-null      int32  
 12  Leather_interior         1 non-null      int32  
 13  Wheel                    1 non-null      int32  
 14  Gear_box_type_Automatic  1 non

In [ ]:
# load model 
with open('XGBRegressor_model.pkl','rb') as file2 : 
    XGBRegressor_model = load(file2)

In [201]:
XGBRegressor_model.predict(df3)   # 13328

array([17426.69], dtype=float32)